# Assignment 5: Data Preprocessing and Feature Engineering

## Dataset: Adult Census Income
Predicts whether income exceeds $50K/yr based on census data.

**Topics Covered:**
- Data Exploration and Preprocessing
- Scaling (Standard & Min-Max)
- Encoding (One-Hot & Label)
- Feature Engineering
- Feature Selection (Isolation Forest, PPS)

---
## Step 1: Import Libraries and Load Data

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder
from sklearn.ensemble import IsolationForest

# Load the dataset
df = pd.read_csv('adult_with_headers.csv')

# Display basic info
print("Dataset loaded successfully!")
print("Shape:", df.shape)
print("\nFirst 5 rows:")
df.head()

---
## Task 1: Data Exploration and Preprocessing

In [ ]:
# Basic data exploration
print("=== Data Types ===")
print(df.dtypes)

print("\n=== Summary Statistics ===")
df.describe()

In [ ]:
# Check for missing values
print("=== Missing Values ===")

# In this dataset, missing values are represented as ' ?' (with space)
# Let's check for this pattern
for column in df.columns:
    # Check for ' ?' pattern
    if df[column].dtype == 'object':
        missing_count = (df[column].str.strip() == '?').sum()
        if missing_count > 0:
            print(column + ":", missing_count, "missing values")

# Also check for regular null values
print("\nNull values:")
print(df.isnull().sum())

In [ ]:
# Handle missing values (replace '?' with NaN then fill with mode)
print("=== Handling Missing Values ===")

# Clean string columns by stripping whitespace
for column in df.columns:
    if df[column].dtype == 'object':
        df[column] = df[column].str.strip()
        # Replace '?' with NaN
        df[column] = df[column].replace('?', np.nan)

# Fill missing values with mode (most frequent value)
for column in df.columns:
    missing_count = df[column].isnull().sum()
    if missing_count > 0:
        mode_value = df[column].mode()[0]
        df[column] = df[column].fillna(mode_value)
        print("Filled", missing_count, "missing values in", column, "with mode:", mode_value)

print("\nRemaining missing values:", df.isnull().sum().sum())

### 1.1 Scaling Techniques

**Standard Scaling:** Transforms data to have mean=0 and std=1
- Best when: Data is normally distributed, algorithm uses distance measures

**Min-Max Scaling:** Transforms data to range [0, 1]
- Best when: You need bounded values, neural networks

In [ ]:
# Identify numerical columns
numerical_columns = ['age', 'fnlwgt', 'education_num', 'capital_gain', 'capital_loss', 'hours_per_week']

print("Original Data (first 5 rows):")
print(df[numerical_columns].head())

In [ ]:
# Standard Scaling
print("=== Standard Scaling ===")

# Create a copy for standard scaling
df_standard = df.copy()

# Apply standard scaling manually (to show the process)
for column in numerical_columns:
    mean_val = df[column].mean()
    std_val = df[column].std()
    df_standard[column + '_std'] = (df[column] - mean_val) / std_val

print("After Standard Scaling:")
std_cols = [col + '_std' for col in numerical_columns]
print(df_standard[std_cols].head())

print("\nVerification (mean should be ~0, std should be ~1):")
for col in std_cols:
    print(col, "- Mean:", round(df_standard[col].mean(), 4), "Std:", round(df_standard[col].std(), 4))

In [ ]:
# Min-Max Scaling
print("=== Min-Max Scaling ===")

# Create a copy for min-max scaling
df_minmax = df.copy()

# Apply min-max scaling manually
for column in numerical_columns:
    min_val = df[column].min()
    max_val = df[column].max()
    df_minmax[column + '_mm'] = (df[column] - min_val) / (max_val - min_val)

print("After Min-Max Scaling:")
mm_cols = [col + '_mm' for col in numerical_columns]
print(df_minmax[mm_cols].head())

print("\nVerification (min should be 0, max should be 1):")
for col in mm_cols:
    print(col, "- Min:", round(df_minmax[col].min(), 4), "Max:", round(df_minmax[col].max(), 4))

**When to use which scaling:**

| Standard Scaling | Min-Max Scaling |
|-----------------|----------------|
| Data is normally distributed | Data needs to be in [0,1] range |
| Algorithms: SVM, Logistic Regression | Algorithms: Neural Networks, KNN |
| Not affected by outliers much | Sensitive to outliers |

---
## Task 2: Encoding Techniques

In [ ]:
# Identify categorical columns
categorical_columns = ['workclass', 'education', 'marital_status', 'occupation', 
                       'relationship', 'race', 'sex', 'native_country', 'income']

# Count unique values in each categorical column
print("=== Unique Values in Categorical Columns ===")
for column in categorical_columns:
    unique_count = df[column].nunique()
    print(column + ":", unique_count, "unique values")

In [ ]:
# Separate columns based on unique values
# Less than 5 categories -> One-Hot Encoding
# 5 or more categories -> Label Encoding

one_hot_columns = []  # < 5 categories
label_encode_columns = []  # >= 5 categories

for column in categorical_columns:
    unique_count = df[column].nunique()
    if unique_count < 5:
        one_hot_columns.append(column)
    else:
        label_encode_columns.append(column)

print("One-Hot Encoding columns (< 5 categories):")
print(one_hot_columns)

print("\nLabel Encoding columns (>= 5 categories):")
print(label_encode_columns)

In [ ]:
# One-Hot Encoding
print("=== One-Hot Encoding ===")

df_encoded = df.copy()

# Apply one-hot encoding using pandas get_dummies
df_encoded = pd.get_dummies(df_encoded, columns=one_hot_columns)

print("Original shape:", df.shape)
print("After One-Hot Encoding:", df_encoded.shape)

# Show new columns created
new_cols = [col for col in df_encoded.columns if col not in df.columns]
print("\nNew columns created:")
for col in new_cols[:10]:
    print(" -", col)

In [ ]:
# Label Encoding
print("=== Label Encoding ===")

# Apply label encoding
for column in label_encode_columns:
    print("\nEncoding:", column)
    print("Original values:", df_encoded[column].unique()[:5], "...")
    
    # Create mapping dictionary
    unique_values = df_encoded[column].unique()
    mapping = {}
    for i in range(len(unique_values)):
        mapping[unique_values[i]] = i
    
    # Apply mapping
    df_encoded[column] = df_encoded[column].map(mapping)
    print("Encoded values:", df_encoded[column].unique()[:5], "...")

print("\nLabel Encoding complete!")

**Pros and Cons:**

| One-Hot Encoding | Label Encoding |
|-----------------|----------------|
| **Pros:** No ordinal assumption | **Pros:** Simple, no extra columns |
| **Pros:** Works with all algorithms | **Pros:** Memory efficient |
| **Cons:** Creates many columns | **Cons:** Implies ordinal relationship |
| **Cons:** Memory intensive | **Cons:** May mislead some algorithms |

---
## Task 3: Feature Engineering

In [ ]:
# Create new features
print("=== Feature Engineering ===")

# Feature 1: Total Capital (gain - loss)
# Rationale: Net capital change is more informative than separate gain/loss
df['total_capital'] = df['capital_gain'] - df['capital_loss']
print("Feature 1: total_capital = capital_gain - capital_loss")
print("This shows the net financial impact of capital transactions.")
print("Sample values:", df['total_capital'].head().tolist())

print()

In [ ]:
# Feature 2: Age Group
# Rationale: Income patterns often differ by age brackets
print("Feature 2: age_group")
print("Rationale: Income patterns often differ by life stages.")

# Create age groups using simple if-else logic
age_groups = []
for age in df['age']:
    if age < 25:
        age_groups.append('Young')
    elif age < 45:
        age_groups.append('Middle')
    elif age < 65:
        age_groups.append('Senior')
    else:
        age_groups.append('Elder')

df['age_group'] = age_groups
print("Age groups created:")
print(df['age_group'].value_counts())

In [ ]:
# Log Transformation on skewed feature
print("=== Log Transformation ===")

# Check skewness of capital_gain
print("Checking skewness of capital_gain:")
print("Mean:", df['capital_gain'].mean())
print("Median:", df['capital_gain'].median())
print("Skewness:", df['capital_gain'].skew())

# The large difference between mean and median indicates right skewness

# Apply log transformation (adding 1 to avoid log(0))
df['capital_gain_log'] = np.log1p(df['capital_gain'])

print("\nAfter log transformation:")
print("New skewness:", df['capital_gain_log'].skew())

# Visualize before and after
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

axes[0].hist(df['capital_gain'], bins=50, color='blue', edgecolor='black')
axes[0].set_title('Before: capital_gain')
axes[0].set_xlabel('Value')

axes[1].hist(df['capital_gain_log'], bins=50, color='green', edgecolor='black')
axes[1].set_title('After: capital_gain_log')
axes[1].set_xlabel('Log Value')

plt.tight_layout()
plt.savefig('log_transformation.png')
plt.show()

---
## Task 4: Feature Selection

### 4.1 Isolation Forest for Outlier Detection

**Isolation Forest** identifies outliers by isolating observations.
Outliers are easier to isolate and thus get shorter path lengths.

In [ ]:
# Isolation Forest for Outlier Detection
print("=== Isolation Forest ===")

# Select numerical features for outlier detection
features_for_outliers = df[numerical_columns].copy()

# Create Isolation Forest model
iso_forest = IsolationForest(contamination=0.05, random_state=42)

# Fit and predict
outlier_labels = iso_forest.fit_predict(features_for_outliers)

# Count outliers (labeled as -1)
outliers_count = 0
inliers_count = 0
for label in outlier_labels:
    if label == -1:
        outliers_count = outliers_count + 1
    else:
        inliers_count = inliers_count + 1

print("Total records:", len(outlier_labels))
print("Outliers detected:", outliers_count)
print("Inliers (normal):", inliers_count)
print("Outlier percentage:", round(outliers_count/len(outlier_labels)*100, 2), "%")

In [ ]:
# Create dataframe without outliers
df['is_outlier'] = outlier_labels
df_clean = df[df['is_outlier'] == 1].copy()

print("Original dataset size:", len(df))
print("After removing outliers:", len(df_clean))

print("\n=== How Outliers Affect Model Performance ===")
print("1. Outliers can skew model training towards extreme values")
print("2. Distance-based algorithms (KNN, SVM) are especially sensitive")
print("3. Outliers can increase model variance and reduce generalization")
print("4. They may represent data errors or rare but valid observations")

### 4.2 Correlation Matrix

In [ ]:
# Correlation Matrix
print("=== Correlation Matrix ===")

# Select numerical columns for correlation
numerical_data = df[numerical_columns + ['total_capital', 'capital_gain_log']]

# Calculate correlation
correlation_matrix = numerical_data.corr()

# Plot heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, fmt='.2f', cmap='coolwarm', center=0)
plt.title('Correlation Matrix')
plt.tight_layout()
plt.savefig('correlation_matrix.png')
plt.show()

### 4.3 PPS (Predictive Power Score)

**PPS** measures how well one variable can predict another.
Unlike correlation, it:
- Works with categorical variables
- Captures non-linear relationships
- Is asymmetric (A->B may differ from B->A)

In [ ]:
# Simple PPS-like analysis using cross-tabulation
# Note: Full PPS requires 'ppscore' library
print("=== Predictive Power Analysis ===")

# Calculate predictive relationships for categorical variables
# Using chi-square test approach

from scipy.stats import chi2_contingency

# Check relationship between education and income
print("\nEducation vs Income:")
contingency_table = pd.crosstab(df['education'], df['income'])
chi2, p_value, dof, expected = chi2_contingency(contingency_table)
print("Chi-square statistic:", round(chi2, 2))
print("P-value:", p_value)
if p_value < 0.05:
    print("Result: Strong relationship exists!")

# Check relationship between occupation and income
print("\nOccupation vs Income:")
contingency_table = pd.crosstab(df['occupation'], df['income'])
chi2, p_value, dof, expected = chi2_contingency(contingency_table)
print("Chi-square statistic:", round(chi2, 2))
print("P-value:", p_value)
if p_value < 0.05:
    print("Result: Strong relationship exists!")

In [ ]:
# Compare findings
print("=== Comparison: Correlation vs PPS ===")
print()
print("Correlation Matrix:")
print("- Only measures LINEAR relationships")
print("- Works only with numerical variables")
print("- Symmetric (A,B) = (B,A)")
print()
print("PPS (Predictive Power Score):")
print("- Measures ANY relationship (linear or non-linear)")
print("- Works with categorical AND numerical variables")
print("- Asymmetric (A predicting B may differ from B predicting A)")
print()
print("Key Finding: Variables like education and occupation")
print("show strong predictive power for income even though")
print("they can't be measured using simple correlation.")

---
## Summary

In this assignment, we learned:

1. **Data Preprocessing:**
   - Handling missing values (imputation with mode)
   - Scaling: Standard (mean=0, std=1) vs Min-Max (range 0-1)

2. **Encoding:**
   - One-Hot: For categorical with few categories
   - Label: For categorical with many categories

3. **Feature Engineering:**
   - Created total_capital (net capital change)
   - Created age_group (categorized ages)
   - Applied log transformation to reduce skewness

4. **Feature Selection:**
   - Isolation Forest to detect and remove outliers
   - Correlation matrix for linear relationships
   - PPS for capturing all types of relationships